# Simple implementation af Kalman filteret
\begin{align*}
    \intertext{kinematic and observations model}
    \mathbf{x}[n]&=\Phi \mathbf{x}[n-1] + \mathbf{u}[n], \\
    \mathbf{z}[n]&=\mathbf{x}[n] + \mathbf{w}[n],\\
    \intertext{the prediction and prediction mean square error,}
    \hat{\mathbf{x}}[n|n-1]&=\Phi\hat{\mathbf{x}}[n-1|n-1], \\
    M[n|n-1]&=\Phi M[n-1|n-1]\Phi^\top + S_{\mathbf{u}}, \\
    \intertext{the Kalman gain,}
    K[n]&=M[n|n-1](S_{\mathbf{w}}+M[n|n-1])^{-1},\\
    \intertext{and the correction and estimate mean square error,}
    \hat{\mathbf{x}}[n|n]&=\hat{\mathbf{x}}[n|n-1]+K[n](\mathbf{z}[n]-\hat{\mathbf{x}}[n|n-1]),\\
    M[n|n]&=(I-K[n])M[n|n-1].
\end{align*}
<br>

In [ ]:
import numpy as np
from itertools import product


def init_gate(q1, q2, dt, vmax=0.5):
    if np.linalg.norm(q1 - q2) <= vmax * dt:
        return True
    else:
        return False

In [ ]:
# simulate data
m01 = np.random.rand(3)
m02 = np.random.rand(3)
m03 = np.random.rand(3)
S0 = [m01, m02, m03]
keylist = list(range(1, len(S0)+1))

tracks = {0: []}
for i in range(len(keylist)):
    tracks[keylist[i]] = [S0[i]]


m11 = np.random.rand(3)
m12 = np.random.rand(3)
m13 = np.random.rand(3)
S1 = [m11, m12, m13]

In [ ]:
def create_hyp_table(S0, S1, tracks, initial_hyp=False):
    # numbered tracks
    current_tracks = np.sort(list(tracks.keys()))
    new_track_start = current_tracks[-1]+1
    track_numbers = np.arange(new_track_start, new_track_start+len(S1))

    # create initial hypothesis table
    hyp_table = []
    for i in range(len(S1)):
        mn_hyp = [0]
        for j in range(len(S0)):
            if init_gate(S0[i], S1[j], 1, 0.75):
                mn_hyp.append(j + 1)
        mn_hyp.append(track_numbers[i])
        hyp_table.append(mn_hyp)

    # create all possible combinations
    combinations = [p for p in product(*hyp_table)]
    perm_table = np.asarray(combinations).T

    # remove impossible combinations
    non_zero_duplicates = []
    for i in range(len(perm_table[0])):
        # if there is a duplicate in column i+1 of perm_table, the value is saved in dup
        u, c = np.unique(perm_table[:, i], return_counts=True)
        dup = u[c > 1]

        # if there are non-zero duplicates, non_zero_duplicates gets a True, otherwise it gets a false
        non_zero_duplicates.append(np.any(dup > 0))

    hyp_possible = np.delete(perm_table, non_zero_duplicates, axis=1)

    return hyp_possible